In [1]:
import random
import os
os.environ["HF_HOME"] = r"./.cache"

from transformers import EncoderDecoderModel, AutoTokenizer, GenerationConfig, Seq2SeqTrainer, Seq2SeqTrainingArguments
from tokenizers import processors
import evaluate

- Encoders
    - BERT_JA : `cl-tohoku/bert-base-japanese-v3`
    - BERT_EN : `bert-base-uncased`, `prajjwal1/bert-tiny`
- Decorders
    - GPT_JA : `rinna/japanese-gpt2-xsmall`
    - GPT_EN : `gpt2`

In [2]:
source_lng = "ja"

if source_lng == "en":
    target_lng = "ja"
    encoder = "bert-base-uncased"
    decoder = "rinna/japanese-gpt2-small"
else: 
    target_lng = "en"
    encoder = "cl-tohoku/bert-base-japanese-v3"
    decoder = "gpt2"

model = EncoderDecoderModel.from_encoder_decoder_pretrained(
    encoder, decoder, encoder_add_pooling_layer=False
)
model.cuda();

Some weights of GPT2LMHeadModel were not initialized from the model checkpoint at gpt2 and are newly initialized: ['h.10.crossattention.c_proj.bias', 'h.10.ln_cross_attn.weight', 'h.10.ln_cross_attn.bias', 'h.0.crossattention.c_attn.weight', 'h.9.ln_cross_attn.bias', 'h.8.crossattention.c_attn.weight', 'h.4.crossattention.c_proj.bias', 'h.2.crossattention.c_attn.bias', 'h.4.crossattention.q_attn.bias', 'h.2.crossattention.c_attn.weight', 'h.9.crossattention.q_attn.weight', 'h.4.crossattention.c_attn.weight', 'h.6.crossattention.c_attn.weight', 'h.7.ln_cross_attn.bias', 'h.1.crossattention.c_proj.bias', 'h.5.crossattention.c_proj.weight', 'h.0.crossattention.q_attn.bias', 'h.11.crossattention.c_proj.weight', 'h.10.crossattention.c_proj.weight', 'h.0.ln_cross_attn.weight', 'h.11.crossattention.c_attn.bias', 'h.9.crossattention.q_attn.bias', 'h.10.crossattention.q_attn.weight', 'h.3.ln_cross_attn.weight', 'h.4.crossattention.c_proj.weight', 'h.7.crossattention.c_proj.weight', 'h.7.ln_cros

In [3]:
def print_model_parameters():
    t_pars, t_bytes = 0, 0
    for p in model.parameters():
        t_pars += p.nelement()
        t_bytes += p.nelement() * p.element_size()

    c_attn_pars, c_attn_bytes = 0, 0
    for layer in model.decoder.transformer.h:
        for p in layer.crossattention.parameters():
            c_attn_pars += p.nelement()
            c_attn_bytes += p.nelement() * p.element_size()
        for p in layer.ln_cross_attn.parameters():
            c_attn_pars += p.nelement()
            c_attn_bytes += p.nelement() * p.element_size()

    print(f"Total number of parameters: {t_pars:12,} ({(t_bytes / 1024**2):7,.1f}MB)")
    print(f"Cross-attention parameters: {c_attn_pars:12,} ({(c_attn_bytes / 1024**2):7,.1f}MB)")

print_model_parameters()

Total number of parameters:  263,423,232 (1,004.9MB)
Cross-attention parameters:   28,366,848 (  108.2MB)


In [4]:
def set_cross_attention_only(model):
    for p in model.parameters():
        p.requires_grad = False
    for layer in model.decoder.transformer.h:
        for p in layer.crossattention.parameters():
            p.requires_grad = True
        for p in layer.ln_cross_attn.parameters():
            p.requires_grad = True
# set_cross_attention_only(model)

In [5]:
encoder_tokenizer = AutoTokenizer.from_pretrained(encoder, use_fast=True)
decoder_tokenizer = AutoTokenizer.from_pretrained(decoder, use_fast=True)
if decoder_tokenizer.pad_token_id is None:
    decoder_tokenizer.pad_token_id = decoder_tokenizer.eos_token_id

model.config.decoder_start_token_id = decoder_tokenizer.bos_token_id
model.config.eos_token_id = decoder_tokenizer.eos_token_id
model.config.pad_token_id = decoder_tokenizer.eos_token_id

# add EOS token at the end of each sentence
decoder_tokenizer._tokenizer.post_processor = processors.TemplateProcessing(
    single="$A " + decoder_tokenizer.eos_token,
    special_tokens=[(decoder_tokenizer.eos_token, decoder_tokenizer.eos_token_id)],
)

In [6]:
from utils.dataset import EnJaDatasetMaker
from transformers import DataCollatorForSeq2Seq

dataset = EnJaDatasetMaker.load_dataset("ja-en-test-1")
train_data = dataset.select(range(100))
valid_data = dataset.select(range(100, 150))

data_collator = DataCollatorForSeq2Seq(encoder_tokenizer, model=model)

In [7]:
metric = evaluate.load("sacrebleu")

def compute_metrics(preds):
    preds_ids, labels_ids = preds

    labels_ids[labels_ids == -100] = decoder_tokenizer.eos_token_id
    references = decoder_tokenizer.batch_decode(labels_ids, skip_special_tokens=True)
    references = [[reference] for reference in references]

    predictions = decoder_tokenizer.batch_decode(preds_ids, skip_special_tokens=True)

    if target_lng == "ja":
        bleu_output = metric.compute(
            references=references, 
            predictions=predictions, 
            tokenize="ja-mecab"
        )
    else:
        bleu_output = metric.compute(
            references=references, 
            predictions=predictions
        )
    return bleu_output


In [8]:
MAX_LENGHT = 128
def set_decoder_configuration(gc: GenerationConfig):
    gc.no_repeat_ngram_size = 3
    gc.length_penalty = 2.0
    gc.num_beams = 3
    #gen_config.max_new_tokens = MAX_LENGHT
    gc.max_length = MAX_LENGHT * 2
    gc.min_length = 0
    gc.early_stopping = True
    gc.pad_token_id = decoder_tokenizer.eos_token_id
    gc.bos_token_id = decoder_tokenizer.bos_token_id
    gc.eos_token_id = decoder_tokenizer.eos_token_id
    return gc

gen_config = GenerationConfig()
gen_config = set_decoder_configuration(gen_config)

In [9]:
train_args = Seq2SeqTrainingArguments(
    report_to="wandb",
    run_name="testing-data-maker-1",
    num_train_epochs=5,

    logging_strategy="steps",
    logging_steps=10,

    evaluation_strategy="epoch",

    output_dir="./.ckp/",
    save_strategy="steps",
    save_steps=1000,
    save_total_limit=4,

    optim="adamw_torch",
    bf16=True,

    per_device_train_batch_size=8,
    gradient_accumulation_steps=1,
    
    group_by_length=True,
    length_column_name="length",

    per_device_eval_batch_size=8,
    predict_with_generate=True,
    generation_config=gen_config,
    # torch_compile=True,
    # label_smoothing_factor=0,
    # auto_find_batch_size=False,
)

In [10]:
trainer = Seq2SeqTrainer(
    model, 
    args=train_args,
    data_collator=data_collator,
    train_dataset=train_data.remove_columns(""), 
    eval_dataset=valid_data, 
    compute_metrics=compute_metrics
)

In [17]:
train_data["source"]

['あまり他人には頼ってはいけない。',
 '私は今年は成績が悪かった。',
 '今日は我々は野宿しないといけない。',
 '彼女はとても美しい。その上、とても賢い。',
 '静かに歩けないのか。',
 'もう一度送ってくれませんか。',
 'タクシーが到着した。',
 'あなたのためならどんなことでもするよ。',
 '通りを歩いていたら、財布を見つけた。',
 '他人には親切であれ。',
 '私と行けば一番いいんだろう。',
 'あの男は目つきが悪い。',
 'トムは歩いて学校へ通っている。',
 '彼の言ったことはいい意見だ。',
 '私たちはローマで楽しく過ごしてます。',
 '学校まで一緒に歩いてくれませんか。',
 '残念ながらあなたを助けることは出来ません。',
 'お釣りが違いますよ。',
 '私はそこへ１度行ったことがある。',
 '彼女は決して馬鹿ではない。',
 '彼が歌っているのが、聞こえますか。',
 '私が忘れたら注意してください。',
 '私のことはおいておいてくれ！',
 'あなたに家にいてもらいたい。',
 '僕があなたの仕事を次にしよう。',
 '子供でも世の中の事を理解する必要がある。',
 '彼女は私に「おはよう」とさえ言わなかった。',
 'この時計は水に強いです。',
 '彼は幸福な生活を送った。',
 'その仕事は私の健康にとって負担だ。',
 'あなたの努力はもうすぐ結果が出るだろう。',
 'これはなんと面白い本でしょう。',
 '彼は私の兄よりも年上に見えます。',
 '彼は彼女のたよりを待ち望んでいる。',
 '彼女のお母さんは前の週の木曜日から病気です。',
 '僕は彼女の歌のピアノを弾いた。',
 'もうベッドにつかなくてはなりません。',
 '英語を書くときは、彼はしばしば辞書を調べる。',
 '行ってしまえ！',
 '数学は難しい科目だ。',
 '１時間経過すれば戻ってきます。',
 '彼が言ったことは正しくないとわかった。',
 'その小説を作った人は誰ですか。',
 '両親は私が外国の学校で勉強をすることに反対した。',
 '彼女はその手紙を日本語からフランス語の翻訳した。',
 '私はいつものように早く起きた。',
 '私を見なさい。',
 'どの電車に乗るのですか。',
 '彼は音楽会が終わるまでに

In [14]:
data_collator(train_data)

ValueError: all the input arrays must have same number of dimensions, but the array at index 0 has 2 dimension(s) and the array at index 1 has 1 dimension(s)

In [11]:
model.train()
trainer.train()

Failed to detect the name of this notebook, you can set it manually with the WANDB_NOTEBOOK_NAME environment variable to enable code saving.
wandb: Currently logged in as: davidboening (dandd). Use `wandb login --relogin` to force relogin


  0%|          | 0/65 [00:00<?, ?it/s]

ValueError: all the input arrays must have same number of dimensions, but the array at index 0 has 2 dimension(s) and the array at index 1 has 1 dimension(s)

In [ ]:
model.cuda()
model.eval()
train_out = trainer.predict(train_data)
valid_out = trainer.predict(valid_data)

print("Train:", compute_metrics((train_out.predictions, train_data["labels"])))
print("Valid:", compute_metrics((valid_out.predictions, valid_data["labels"])))

In [ ]:
train_decode = decoder_tokenizer.batch_decode(train_out.predictions, skip_special_tokens=True)
valid_decode = decoder_tokenizer.batch_decode(valid_out.predictions, skip_special_tokens=True)

In [ ]:
def print_pairs(dataset, generation, sample=5):
    assert len(dataset) == len(generation), "Invalid combination!"

    sample_ids = random.sample(range(len(dataset)), sample)
    for i, sid in enumerate(sample_ids):
        print(f"Sentence #{i} [id={sid}]")
        print(
            f"\tOriginal:  {dataset['source'][sid]}\n"
            f"\tTarget:    {dataset['target'][sid]}\n"
            f"\tGenerated: {generation[sid]}\n"
        )
    return

print_pairs(train_data, train_decode, sample=3)